## Setup

In [43]:
import requests
import os
from getpass import getpass
from bs4 import BeautifulSoup
import pandas as pd
import json
import ast
from tqdm.auto import tqdm
from IPython.display import display, Markdown
from datasets import Dataset

os.environ["GITHUB_TOKEN"] = getpass("")

## Collect issue urls

In [6]:
URL = "https://github.com/rust-lang/rust/issues?page={}&q=is%3Aopen+is%3Aissue+label%3AS-tracking-needs-summary"
issue_url_collector = []

for page_num in tqdm(range(1, 4)):
    response = requests.get(URL.format(page_num))
    soup = BeautifulSoup(response.content, "html.parser")
    urls = soup.find_all("a", class_="Link--primary v-align-middle no-underline h4 js-navigation-open markdown-title")
    issue_urls = ["https://api.github.com/repos/rust-lang/rust/issues/" + x["href"].split("/")[-1] for x in urls]
    issue_url_collector.extend(issue_urls)

  0%|          | 0/3 [00:00<?, ?it/s]

## Collect issue main content

In [13]:
headers = {"Authorization": f"token {os.getenv('GITHUB_TOKEN')}"}
issue_data = []

for issue_url in tqdm(issue_url_collector):
    response = requests.get(issue_url, headers=headers)
    issue = response.json()
    issue_short = {k: v for k, v in issue.items() if k in ["url", "comments_url", "html_url"]}
    issue_short["labels"] = "<|endoftext|>".join([x["name"] for x in issue["labels"]])
    issue_data.append(issue_short)

  0%|          | 0/52 [00:00<?, ?it/s]

In [14]:
df = pd.DataFrame.from_dict(issue_data)

In [15]:
df.head()

,url,comments_url,html_url,labels
0,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/97362,T-lang<|endoftext|>C-tracking-issue<|endoftext...
1,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/92827,A-associated-items<|endoftext|>T-lang<|endofte...
2,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/91971,T-lang<|endoftext|>T-libs-api<|endoftext|>C-tr...
3,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/89460,A-lint<|endoftext|>T-lang<|endoftext|>T-compil...
4,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/86555,B-RFC-approved<|endoftext|>T-lang<|endoftext|>...


## Collect issue comments

In [16]:
def get_comments(url):
    response = requests.get(url, headers=headers)
    json_content = response.json()
    comments = "<|endoftext|>".join([x["body"] for x in json_content])

    return comments

df["comments"] = df["comments_url"].apply(get_comments)

In [17]:
df.head()

,url,comments_url,html_url,labels,comments
0,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/97362,T-lang<|endoftext|>C-tracking-issue<|endoftext...,I'd like to work on implementing this 👀 \r\n\r...
1,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/92827,A-associated-items<|endoftext|>T-lang<|endofte...,"Hello!\r\nIs there a corresponding RFC, or any..."
2,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/91971,T-lang<|endoftext|>T-libs-api<|endoftext|>C-tr...,Context for labels applied: it sounds like we ...
3,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/89460,A-lint<|endoftext|>T-lang<|endoftext|>T-compil...,Could we get a more detailed summary of the ca...
4,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/86555,B-RFC-approved<|endoftext|>T-lang<|endoftext|>...,It looks like this has been implemented; are t...


## Check

In [37]:
template = """
************************************************************************************
************************************************************************************
************************************************************************************

URL: {url}\n
URL COMMENTS: {comments_url}\n
URL HTML: {html_url}\n
LABELS: {labels}\n
COMMENTS:\n
{comments}
\n\n
""".strip()

for i, row in df[:3].iterrows():
    labels = row["labels"].split("<|endoftext|>")
    comments = row["comments"].replace("<|endoftext|>", "\n")
    display(Markdown(template.format(
        url=row["url"],
        comments_url=row["comments_url"],
        html_url=row["html_url"],
        labels=labels,
        comments=comments
        ))
    )

************************************************************************************
************************************************************************************
************************************************************************************

URL: https://api.github.com/repos/rust-lang/rust/issues/97362

URL COMMENTS: https://api.github.com/repos/rust-lang/rust/issues/97362/comments

URL HTML: https://github.com/rust-lang/rust/issues/97362

LABELS: ['T-lang', 'C-tracking-issue', 'S-tracking-needs-summary', 'T-types', 'F-closure_lifetime_binder']

COMMENTS:

I'd like to work on implementing this 👀 

@rustbot claim
The implementation section needs to be updated to mention #98705.
I bumped on this issue and I would like to share some code that maybe someone knowledgeable could comment on whether we are likely to get the features demonstrated or similar (in summary it's getting `for<>` for async closures):

```rust
#![feature(closure_lifetime_binder)]
#![feature(trait_alias)]

use core::future::Ready;
use core::future::ready;
use core::future::Future;

// Don't read too much into the implementation of `S`
struct S<'a>(&'a ());

trait AsyncF<'a, F> = Fn(S<'a>) -> F where F: Future<Output = S<'a>>;
async fn asyncF<F>(f: impl for<'a> AsyncF<'a, F>) {
    let a = ();
    f(S(&a)).await;
}

fn main() {
    // We can't seem to do this because for<> does not seem to extend to the
    // argument.
    asyncF(for<'a> |x: S<'a>| -> Ready<S<'a>> {ready(x)});

    
    // Ideally we would like this to do anything useful with for<> and async.
    // asyncF(for<'a> async move |x: &'a ()| -> &'a () {ready(x)});
}
```
[playground](https://play.rust-lang.org/?version=nightly&mode=debug&edition=2021&gist=75475a141e376a2c0711ecac8461d20e)
@fakedrake I think your `AsyncF`/`asyncF` are wrong. You are using a single `F` type, that is supposed to implement `for<'a> Future<Output = S<'a>>`, where in reality you have a `for<'a> F<'a>: Future<Output = S<'a>>` type.

If you change the `AsyncF` trait to be something like the following:
```rust
trait AsyncF<'a, Input>: Fn(Input) -> Self::Future {
    type Future: Future<Output = Self::Out>;
    
    type Out;
}

impl<'a, F, Fut, In> AsyncF<'a, In> for F
where
    F: Fn(In) -> Fut,
    Fut: Future,
{
    type Future = Fut;
    type Out = Fut::Output;
}
```
Then your example [works](https://play.rust-lang.org/?version=nightly&mode=debug&edition=2021&gist=a0db8d120af26cbcab4e20dc309b4adc). Though I want to warn you that such traits are a pain to work with and especially extend.

----

As for `for<'a> async move |x: &'a ()| -> &'a ()` kind of thing, I think it generally desired, but was ruled out of the scope of the RFC. So, if we'll ever add it, it would first need its own RFC.

It appears that async closures are currently in a very rough shape ([eg](https://github.com/rust-lang/rust/blob/b8c0a01b2b651416f5e7461209ff1a93a98619e4/compiler/rustc_ast_lowering/src/expr.rs#L962)) so a good place to start would probably be to fix/complete the implementation of async closures first.
Thank you very much. I have seen people vaguely complaining about the primitive state of async closures but besides this niche case I have found that they have worked pretty well. Is there an issue tracking the problems with them?
There is a tracking issue for async closures: #62290
Found an ICE involving this feature: https://github.com/rust-lang/rust/issues/103736

(Maybe we could use an `F-closure_lifetime_binder` tag?)
Suggestion: could it be allowed to elide the return type of an explicitly higher-ranked closure when it is `()`? Currently, Rust allows eliding `()` returns in all other function signatures, even when nothing else can be elided.
@Jules-Bertholet I don't think it's a good idea, we may want to allow inferring return type in the future, just like with normal closures.
@WaffleLapkin "Infer return type when it is `()`" should be forward-compatible with "infer return type always."
@Jules-Bertholet oh, I misunderstood this as "make `for<> || { some expression... }` return type always `()`, same as with normal function". Inferring it is probably fine.
It appears that this will not work well for returning closures as their type can not be explicitly specified as a closure return type. This is likely known but I think worth pointing it out. This will however work for return types that you can specify.

```rust
let f = for<'a> |a: &'a u8| -> impl 'a + Fn() -> &'a u8 {
    || a
};
```

```
error[[E0562]](https://doc.rust-lang.org/nightly/error_codes/E0562.html): `impl Trait` only allowed in function and inherent method return types, not in closure return types
 --> src/main.rs:5:36
  |
5 |     let f = for<'a> |a: &'a u8| -> impl 'a + Fn() -> &'a u8 {
  |                                    ^^^^^^^^^^^^^^^^^^^^^^^^
```

[Playground](https://play.rust-lang.org/?version=nightly&mode=debug&edition=2021&gist=7aa83bd601924e751507bc7062715256)
@kevincox: I believe that is a general limitation of what return types you can write when declaring closures, and not related to this RFC.

************************************************************************************
************************************************************************************
************************************************************************************

URL: https://api.github.com/repos/rust-lang/rust/issues/92827

URL COMMENTS: https://api.github.com/repos/rust-lang/rust/issues/92827/comments

URL HTML: https://github.com/rust-lang/rust/issues/92827

LABELS: ['A-associated-items', 'T-lang', 'C-tracking-issue', 'A-const-generics', 'S-tracking-needs-summary', 'F-associated_const_equality']

COMMENTS:

Hello!
Is there a corresponding RFC, or any context whatsoever concerning those changes ? 
This is the first occurrence of language syntax change that is not backed by RFC and community feedback
@Centril @oli-obk 
Well, we have https://github.com/rust-lang/rust/issues/70256 which indeed has basically no discussion of the feature. It just seemed like an oversight to have associated types + associated type bounds, but only associated consts without any bounds for them. 
I probably should've tagged the original issue in this at the start (it was in the original PR if I remember correctly) but I'm bad at leaving a thorough paper trail. I'll add a reference to the original issue in the issue itself, and if there's more work that needs to be done please let know
You did everything correctly. I found the issue because you did link it in the first sentence in this issue 😄

We should probably do a lang team MCP for it along with some medium sized doc explaining what is going on and why. I'll open a hackmd and then we can collab on it
What's the current state of this? How much is implemented, and how functional is it?
@joshtriplett I've started work on it, and the syntax/parsing is in place, but the implementation currently does not work. I started working on Chalk, but that has stalled since I wasn't really able to know whether it was necessary for implementing it in rustc. I've also gotten a bit sidetracked working on other things.
Is it planned to recognize Associated Constant Equality as disjoint cases?
```rs
trait Foo{
    const DECIDER: bool;
}
trait Bar{ /*stuff*/ }
impl<T: Foo<DECIDER = true>> Bar for T{ /*stuff*/ }
impl<T: Foo<DECIDER = false>> Bar for T{ /*stuff*/ }
```
@rustbot label F-associated_const_equality
@urben1680 I believe this would be the main point of this, but I have not got around to actually implementing it
```rust
pub struct ConstNum<const N: usize>;

pub trait Is<const N: usize> {}

impl<const N: usize> Is<N> for ConstNum<N> {}


struct Bar<const N: usize>([u8; N]);

fn foo<const N: usize>(_input: Bar<N>) where ConstNum<N>: Is<2> {
    
}

fn main() {
    foo(Bar([0u8; 2])); //works
    foo(Bar([0u8; 3])); //doesn't
}
```
Another use case would be checking tensor shapes. I hit this issue when compiling my code that constrains the shapes of input tensors to have the same dimensionality and/or rank. 

For example, dumb code like the below
```rust
pub trait TensorShape1D {
    const SHAPE: usize;  // i.e. length
}

pub fn add<const L: usize, LEFT_TENSOR: TensorShape1D<SHAPE=L>, RIGHT_TENSOR: TensorShape1D<SHAPE=L>>(left: LEFT_TENSOR, right: RIGHT_TENSOR) {
    todo!()
}
```
Any two structs that implement `TensorShape1D` and have the same `SHAPE` can be the input of this function.

Right now, I have to use `typenum` crate and associated type to do the same checking, but it would be great to not have to type-dance like `typenum`. I think when this is implemented, `typenum` can be greatly simplified.
I am locking this, as it is a tracking issue. Please open a thread on zulip for discussions or open issues for any issues you encounter.

************************************************************************************
************************************************************************************
************************************************************************************

URL: https://api.github.com/repos/rust-lang/rust/issues/91971

URL COMMENTS: https://api.github.com/repos/rust-lang/rust/issues/91971/comments

URL HTML: https://github.com/rust-lang/rust/issues/91971

LABELS: ['T-lang', 'T-libs-api', 'C-tracking-issue', 'A-intrinsics', 'S-tracking-design-concerns', 'S-tracking-needs-summary']

COMMENTS:

Context for labels applied: it sounds like we need a summary of what precisely this means, as distinct from `align_of`, in a way that isn't tied to the LLVM backend.

## Upload on Hugging Face Hub

In [41]:
from huggingface_hub import login

login()

In [46]:
hfdf = Dataset.from_pandas(df)
hfdf

Dataset({
    features: ['url', 'comments_url', 'html_url', 'labels', 'comments'],
    num_rows: 52
})

In [53]:
hfdf[0]

{'url': 'https://api.github.com/repos/rust-lang/rust/issues/97362',
 'comments_url': 'https://api.github.com/repos/rust-lang/rust/issues/97362/comments',
 'html_url': 'https://github.com/rust-lang/rust/issues/97362',
 'labels': 'T-lang<|endoftext|>C-tracking-issue<|endoftext|>S-tracking-needs-summary<|endoftext|>T-types<|endoftext|>F-closure_lifetime_binder',
 'comments': 'I\'d like to work on implementing this 👀 \r\n\r\n@rustbot claim<|endoftext|>The implementation section needs to be updated to mention #98705.<|endoftext|>I bumped on this issue and I would like to share some code that maybe someone knowledgeable could comment on whether we are likely to get the features demonstrated or similar (in summary it\'s getting `for<>` for async closures):\r\n\r\n```rust\r\n#![feature(closure_lifetime_binder)]\r\n#![feature(trait_alias)]\r\n\r\nuse core::future::Ready;\r\nuse core::future::ready;\r\nuse core::future::Future;\r\n\r\n// Don\'t read too much into the implementation of `S`\r\nstr

In [55]:
hfdf.push_to_hub("matteopilotto/rust-github-issues")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

## Load dataset from the Hub

In [58]:
from datasets import load_dataset

hfdf_loaded = load_dataset("matteopilotto/rust-github-issues")
hfdf_loaded

DatasetDict({
    train: Dataset({
        features: ['url', 'comments_url', 'html_url', 'labels', 'comments'],
        num_rows: 52
    })
})

In [59]:
df_loaded = hfdf_loaded["train"].to_pandas()

In [60]:
df_loaded.head()

,url,comments_url,html_url,labels,comments
0,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/97362,T-lang<|endoftext|>C-tracking-issue<|endoftext...,I'd like to work on implementing this 👀 \r\n\r...
1,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/92827,A-associated-items<|endoftext|>T-lang<|endofte...,"Hello!\r\nIs there a corresponding RFC, or any..."
2,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/91971,T-lang<|endoftext|>T-libs-api<|endoftext|>C-tr...,Context for labels applied: it sounds like we ...
3,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/89460,A-lint<|endoftext|>T-lang<|endoftext|>T-compil...,Could we get a more detailed summary of the ca...
4,https://api.github.com/repos/rust-lang/rust/is...,https://api.github.com/repos/rust-lang/rust/is...,https://github.com/rust-lang/rust/issues/86555,B-RFC-approved<|endoftext|>T-lang<|endoftext|>...,It looks like this has been implemented; are t...


## Check data loaded from the Hub

In [62]:
template = """
************************************************************************************
************************************************************************************
************************************************************************************

URL: {url}\n
URL COMMENTS: {comments_url}\n
URL HTML: {html_url}\n
LABELS: {labels}\n
COMMENTS:\n
{comments}
\n\n
""".strip()

for i, row in df_loaded[:3].iterrows():
    labels = row["labels"].split("<|endoftext|>")
    comments = row["comments"].replace("<|endoftext|>", "\n")
    display(Markdown(template.format(
        url=row["url"],
        comments_url=row["comments_url"],
        html_url=row["html_url"],
        labels=labels,
        comments=comments
        ))
    )

************************************************************************************
************************************************************************************
************************************************************************************

URL: https://api.github.com/repos/rust-lang/rust/issues/97362

URL COMMENTS: https://api.github.com/repos/rust-lang/rust/issues/97362/comments

URL HTML: https://github.com/rust-lang/rust/issues/97362

LABELS: ['T-lang', 'C-tracking-issue', 'S-tracking-needs-summary', 'T-types', 'F-closure_lifetime_binder']

COMMENTS:

I'd like to work on implementing this 👀 

@rustbot claim
The implementation section needs to be updated to mention #98705.
I bumped on this issue and I would like to share some code that maybe someone knowledgeable could comment on whether we are likely to get the features demonstrated or similar (in summary it's getting `for<>` for async closures):

```rust
#![feature(closure_lifetime_binder)]
#![feature(trait_alias)]

use core::future::Ready;
use core::future::ready;
use core::future::Future;

// Don't read too much into the implementation of `S`
struct S<'a>(&'a ());

trait AsyncF<'a, F> = Fn(S<'a>) -> F where F: Future<Output = S<'a>>;
async fn asyncF<F>(f: impl for<'a> AsyncF<'a, F>) {
    let a = ();
    f(S(&a)).await;
}

fn main() {
    // We can't seem to do this because for<> does not seem to extend to the
    // argument.
    asyncF(for<'a> |x: S<'a>| -> Ready<S<'a>> {ready(x)});

    
    // Ideally we would like this to do anything useful with for<> and async.
    // asyncF(for<'a> async move |x: &'a ()| -> &'a () {ready(x)});
}
```
[playground](https://play.rust-lang.org/?version=nightly&mode=debug&edition=2021&gist=75475a141e376a2c0711ecac8461d20e)
@fakedrake I think your `AsyncF`/`asyncF` are wrong. You are using a single `F` type, that is supposed to implement `for<'a> Future<Output = S<'a>>`, where in reality you have a `for<'a> F<'a>: Future<Output = S<'a>>` type.

If you change the `AsyncF` trait to be something like the following:
```rust
trait AsyncF<'a, Input>: Fn(Input) -> Self::Future {
    type Future: Future<Output = Self::Out>;
    
    type Out;
}

impl<'a, F, Fut, In> AsyncF<'a, In> for F
where
    F: Fn(In) -> Fut,
    Fut: Future,
{
    type Future = Fut;
    type Out = Fut::Output;
}
```
Then your example [works](https://play.rust-lang.org/?version=nightly&mode=debug&edition=2021&gist=a0db8d120af26cbcab4e20dc309b4adc). Though I want to warn you that such traits are a pain to work with and especially extend.

----

As for `for<'a> async move |x: &'a ()| -> &'a ()` kind of thing, I think it generally desired, but was ruled out of the scope of the RFC. So, if we'll ever add it, it would first need its own RFC.

It appears that async closures are currently in a very rough shape ([eg](https://github.com/rust-lang/rust/blob/b8c0a01b2b651416f5e7461209ff1a93a98619e4/compiler/rustc_ast_lowering/src/expr.rs#L962)) so a good place to start would probably be to fix/complete the implementation of async closures first.
Thank you very much. I have seen people vaguely complaining about the primitive state of async closures but besides this niche case I have found that they have worked pretty well. Is there an issue tracking the problems with them?
There is a tracking issue for async closures: #62290
Found an ICE involving this feature: https://github.com/rust-lang/rust/issues/103736

(Maybe we could use an `F-closure_lifetime_binder` tag?)
Suggestion: could it be allowed to elide the return type of an explicitly higher-ranked closure when it is `()`? Currently, Rust allows eliding `()` returns in all other function signatures, even when nothing else can be elided.
@Jules-Bertholet I don't think it's a good idea, we may want to allow inferring return type in the future, just like with normal closures.
@WaffleLapkin "Infer return type when it is `()`" should be forward-compatible with "infer return type always."
@Jules-Bertholet oh, I misunderstood this as "make `for<> || { some expression... }` return type always `()`, same as with normal function". Inferring it is probably fine.
It appears that this will not work well for returning closures as their type can not be explicitly specified as a closure return type. This is likely known but I think worth pointing it out. This will however work for return types that you can specify.

```rust
let f = for<'a> |a: &'a u8| -> impl 'a + Fn() -> &'a u8 {
    || a
};
```

```
error[[E0562]](https://doc.rust-lang.org/nightly/error_codes/E0562.html): `impl Trait` only allowed in function and inherent method return types, not in closure return types
 --> src/main.rs:5:36
  |
5 |     let f = for<'a> |a: &'a u8| -> impl 'a + Fn() -> &'a u8 {
  |                                    ^^^^^^^^^^^^^^^^^^^^^^^^
```

[Playground](https://play.rust-lang.org/?version=nightly&mode=debug&edition=2021&gist=7aa83bd601924e751507bc7062715256)
@kevincox: I believe that is a general limitation of what return types you can write when declaring closures, and not related to this RFC.

************************************************************************************
************************************************************************************
************************************************************************************

URL: https://api.github.com/repos/rust-lang/rust/issues/92827

URL COMMENTS: https://api.github.com/repos/rust-lang/rust/issues/92827/comments

URL HTML: https://github.com/rust-lang/rust/issues/92827

LABELS: ['A-associated-items', 'T-lang', 'C-tracking-issue', 'A-const-generics', 'S-tracking-needs-summary', 'F-associated_const_equality']

COMMENTS:

Hello!
Is there a corresponding RFC, or any context whatsoever concerning those changes ? 
This is the first occurrence of language syntax change that is not backed by RFC and community feedback
@Centril @oli-obk 
Well, we have https://github.com/rust-lang/rust/issues/70256 which indeed has basically no discussion of the feature. It just seemed like an oversight to have associated types + associated type bounds, but only associated consts without any bounds for them. 
I probably should've tagged the original issue in this at the start (it was in the original PR if I remember correctly) but I'm bad at leaving a thorough paper trail. I'll add a reference to the original issue in the issue itself, and if there's more work that needs to be done please let know
You did everything correctly. I found the issue because you did link it in the first sentence in this issue 😄

We should probably do a lang team MCP for it along with some medium sized doc explaining what is going on and why. I'll open a hackmd and then we can collab on it
What's the current state of this? How much is implemented, and how functional is it?
@joshtriplett I've started work on it, and the syntax/parsing is in place, but the implementation currently does not work. I started working on Chalk, but that has stalled since I wasn't really able to know whether it was necessary for implementing it in rustc. I've also gotten a bit sidetracked working on other things.
Is it planned to recognize Associated Constant Equality as disjoint cases?
```rs
trait Foo{
    const DECIDER: bool;
}
trait Bar{ /*stuff*/ }
impl<T: Foo<DECIDER = true>> Bar for T{ /*stuff*/ }
impl<T: Foo<DECIDER = false>> Bar for T{ /*stuff*/ }
```
@rustbot label F-associated_const_equality
@urben1680 I believe this would be the main point of this, but I have not got around to actually implementing it
```rust
pub struct ConstNum<const N: usize>;

pub trait Is<const N: usize> {}

impl<const N: usize> Is<N> for ConstNum<N> {}


struct Bar<const N: usize>([u8; N]);

fn foo<const N: usize>(_input: Bar<N>) where ConstNum<N>: Is<2> {
    
}

fn main() {
    foo(Bar([0u8; 2])); //works
    foo(Bar([0u8; 3])); //doesn't
}
```
Another use case would be checking tensor shapes. I hit this issue when compiling my code that constrains the shapes of input tensors to have the same dimensionality and/or rank. 

For example, dumb code like the below
```rust
pub trait TensorShape1D {
    const SHAPE: usize;  // i.e. length
}

pub fn add<const L: usize, LEFT_TENSOR: TensorShape1D<SHAPE=L>, RIGHT_TENSOR: TensorShape1D<SHAPE=L>>(left: LEFT_TENSOR, right: RIGHT_TENSOR) {
    todo!()
}
```
Any two structs that implement `TensorShape1D` and have the same `SHAPE` can be the input of this function.

Right now, I have to use `typenum` crate and associated type to do the same checking, but it would be great to not have to type-dance like `typenum`. I think when this is implemented, `typenum` can be greatly simplified.
I am locking this, as it is a tracking issue. Please open a thread on zulip for discussions or open issues for any issues you encounter.

************************************************************************************
************************************************************************************
************************************************************************************

URL: https://api.github.com/repos/rust-lang/rust/issues/91971

URL COMMENTS: https://api.github.com/repos/rust-lang/rust/issues/91971/comments

URL HTML: https://github.com/rust-lang/rust/issues/91971

LABELS: ['T-lang', 'T-libs-api', 'C-tracking-issue', 'A-intrinsics', 'S-tracking-design-concerns', 'S-tracking-needs-summary']

COMMENTS:

Context for labels applied: it sounds like we need a summary of what precisely this means, as distinct from `align_of`, in a way that isn't tied to the LLVM backend.